In [95]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable = True)
from tensorflow import keras
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Dropout, Layer, SimpleRNN, LSTM, Flatten
from tensorflow.keras.layers import Embedding, Input, GlobalAveragePooling1D, Dense
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential, Model
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)


In [96]:
print(physical_devices)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [97]:
import os
import re
import numpy as np
import pandas as pd

# text treatement
import nltk
from nltk.tokenize import word_tokenize
import string
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re

nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

all_stopwords = stopwords.words('english')
all_stopwords.remove('not')


import csv,sys
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import svm 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_selection import SelectKBest,chi2 


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [98]:
dataset_train = pd.read_csv('project2_training_data.txt', delimiter= '\r\n' , header = None, names=['text'])
dataset_labels = pd.read_csv('project2_training_data_labels.txt', delimiter= '\r\n' , header = None, names=['text'])

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [99]:
dataset_train.head()

,text
0,Merrill Lynch analyst Campbell Morgan upgraded...
1,Eriikka S+Âderstr+Âm has previously held sever...
2,The webcast may be followed online on the comp...
3,"Typical end-uses include roof structures , flo..."
4,The sale will be finalized in September or Oct...


In [100]:
dataset_train = dataset_train.dropna(axis='columns')

In [101]:
dataset_train.head()

,text
0,Merrill Lynch analyst Campbell Morgan upgraded...
1,Eriikka S+Âderstr+Âm has previously held sever...
2,The webcast may be followed online on the comp...
3,"Typical end-uses include roof structures , flo..."
4,The sale will be finalized in September or Oct...


In [102]:
dataset_labels.head()

,text
0,positive
1,neutral
2,neutral
3,neutral
4,neutral


In [103]:
result = pd.concat([dataset_train, dataset_labels], axis=1, join='inner')

In [104]:
result.head()

,text,text
0,Merrill Lynch analyst Campbell Morgan upgraded...,positive
1,Eriikka S+Âderstr+Âm has previously held sever...,neutral
2,The webcast may be followed online on the comp...,neutral
3,"Typical end-uses include roof structures , flo...",neutral
4,The sale will be finalized in September or Oct...,neutral


In [105]:
result.columns = ['Texts', 'Sentiment']

In [106]:
result.head()

,Texts,Sentiment
0,Merrill Lynch analyst Campbell Morgan upgraded...,positive
1,Eriikka S+Âderstr+Âm has previously held sever...,neutral
2,The webcast may be followed online on the comp...,neutral
3,"Typical end-uses include roof structures , flo...",neutral
4,The sale will be finalized in September or Oct...,neutral


In [107]:
df = result.copy()

In [108]:
df.head()

,Texts,Sentiment
0,Merrill Lynch analyst Campbell Morgan upgraded...,positive
1,Eriikka S+Âderstr+Âm has previously held sever...,neutral
2,The webcast may be followed online on the comp...,neutral
3,"Typical end-uses include roof structures , flo...",neutral
4,The sale will be finalized in September or Oct...,neutral


In [109]:
# Importing LabelEncoder from Sklearn
# library from preprocessing Module.
from sklearn.preprocessing import LabelEncoder
 
# Creating a instance of label Encoder.
le = LabelEncoder()
 
# Using .fit_transform function to fit label
# encoder and return encoded label
label = le.fit_transform(df['Sentiment'])
 
# printing label
label

array([2, 1, 1, ..., 1, 1, 1])

In [110]:

# removing the column 'Purchased' from df
# as it is of no use now.
# df.drop("Sentiment", axis=1, inplace=True)
 
# Appending the array to our dataFrame
# with column name 'Purchased'
df["Class"] = label

# removing special characters
df = df.replace('\*', ' ')
df = df.replace('\xc2', ' ', regex=True)
df = df.replace('\xf1', ' ', regex=True)
df = df.replace('\xc1', ' ', regex=True)
df = df.replace('[^a-zA-Z]', ' ')
# df = df.replace('"+"', ' ', regex=True)
# df = df.replace('"+"', ' ', regex=True)

# printing Dataframe
df

,Texts,Sentiment,Class
0,Merrill Lynch analyst Campbell Morgan upgraded...,positive,2
1,Eriikka S+ derstr+ m has previously held sever...,neutral,1
2,The webcast may be followed online on the comp...,neutral,1
3,"Typical end-uses include roof structures , flo...",neutral,1
4,The sale will be finalized in September or Oct...,neutral,1
...,...,...,...
1806,With this appointment Kaupthing Bank aims to f...,positive,2
1807,Jon Risfelt has previously held operational ex...,neutral,1
1808,The group intends to relocate warehouse and of...,neutral,1
1809,"The contract includes software licences , appl...",neutral,1


In [111]:
max_len = int(df['Texts'].str.len().max())

In [112]:
max_len

315

In [113]:
df["Texts"]= df["Texts"].str.pad(301, side ='both')

In [114]:
df['Class'].unique()

#1 for neutral
#2 for positive
#0 for negative

array([2, 1, 0])

In [115]:
df = df.dropna()

In [116]:
df.rename(columns = {'Texts':'text', 'Class':'target'}, inplace = True)

In [117]:
df.head()

,text,Sentiment,target
0,...,positive,2
1,Eriikk...,neutral,1
2,...,neutral,1
3,Typical end-uses i...,neutral,1
4,...,neutral,1


In [118]:
from keras.utils import to_categorical

In [119]:
one_hot_encode = to_categorical(df.target)
print(one_hot_encode)
y = one_hot_encode

[[0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [120]:
class TransformerBlock(Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = Sequential(
            [Dense(ff_dim, activation="relu"), 
             Dense(embed_dim),]
        )
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.rate = rate

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

    def get_config(self):
      config = super().get_config().copy()
      config.update({'embed_dim': self.embed_dim, 'num_heads': self.num_heads, 'ff_dim': self.ff_dim, 'rate': self.rate})
      return config

In [121]:
class TokenAndPositionEmbedding(Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = Embedding(input_dim=maxlen, output_dim=embed_dim)
        
    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [122]:
vocab_size = 1900  # Only consider the top approx 2k words
maxlen = 200  # Only consider the first 200 words of each movie review

X = df.text

# Split original dataframe into train and temp dataframes.
df_train, df_test, y_train, y_test = train_test_split(X,
                                                      y,
                                                      stratify=y,
                                                      test_size=(0.3))

print(len(df_train), "Training sequences")
print(len(df_test), "Test sequences")

1267 Training sequences
544 Test sequences


In [123]:
vocab_size = 1900  # Only consider the top approx 2k words
maxlen = 200  # Only consider the first 200 words of each movie review

X = df.text

# Split original dataframe into train and temp dataframes.
df_train, df_test, y_train, y_test = train_test_split(X,
                                                      y,
                                                      stratify=y,
                                                      test_size=(0.3))

print(len(df_train), "Training sequences")
print(len(df_test), "Test sequences")

1267 Training sequences
544 Test sequences


In [124]:
from keras.preprocessing.text import Tokenizer
from collections import Counter

def counter_func(string_input):
  var1 = Counter()
   
  for line in string_input.values:
    for word in line.split():
      var1[word] += 1
  return var1

In [125]:
var2 = counter_func(df.text)

In [126]:
num_words = len(var2)
max_len_vec = 0
for line in df.text.values:
  if (len(line.split()) > (max_len_vec)):
    max_len_vec = len(line.split())
  
print(max_len_vec)

81


In [127]:
var3 = Tokenizer(num_words = num_words)
var3.fit_on_texts(df_train)
word_ind = var3.word_index

df_train = var3.texts_to_sequences(df_train)
df_test = var3.texts_to_sequences(df_test)

print(df_train)

[[25, 1005, 149, 126, 476, 63, 34, 16, 199, 90, 46, 56, 21, 20, 14, 72, 5, 1006, 30, 28, 52, 1979, 17, 28, 8, 1, 58, 179, 2, 46, 9, 647, 17, 28, 52, 1326, 17, 28, 8, 1, 102, 35, 2, 48], [1, 35, 11, 26, 477, 5, 6, 90, 42, 22, 9, 6, 555, 31, 22, 150, 5, 1, 478, 137, 358, 97], [804, 805, 648, 1980, 1981, 1007, 556, 1, 1982, 2, 806, 1983, 2, 1984, 1985, 212, 1, 1986, 1987, 4, 479, 1327, 16, 180, 649, 101, 1988, 1989, 1328, 11, 1990, 1991, 16, 807], [25, 272, 4, 1329, 200, 164, 405, 808, 63, 34, 16, 116, 40, 46, 56, 1, 1008, 26, 2, 21, 1330, 84, 157, 19, 809, 251, 23, 16, 23, 5, 31, 42, 28, 52, 24, 31, 28, 8, 1, 273, 38, 2, 48], [810, 811, 44, 130, 50, 64, 5, 130, 17, 3, 1, 359, 23, 35], [1992, 1993, 1994, 1995, 1996, 51, 73, 12, 406, 19, 557, 11, 1009, 1997, 1010, 21, 69, 54, 5, 138, 360, 71, 9, 252, 1011, 71, 18, 361, 26, 8, 1, 1998, 139, 650, 19, 151, 24, 71, 5, 213, 24, 22, 301], [480, 11, 54, 12, 1331, 16, 1, 812, 253, 85], [558, 88, 1332, 1999, 559, 4, 188, 407, 812, 253, 85, 302, 201

In [128]:
df_train = tf.keras.preprocessing.sequence.pad_sequences(df_train, maxlen=max_len_vec)
df_test = tf.keras.preprocessing.sequence.pad_sequences(df_test, maxlen=max_len_vec)

In [129]:
df_train.shape, y_train.shape

((1267, 81), (1267, 3))

In [130]:
df_test.shape, y_test.shape

((544, 81), (544, 3))

In [131]:
y_train

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)

In [132]:
y_train = y_train.astype('int32')
y_test = y_test.astype('int32')

## Choice thingy

In [133]:
choice = int(input("Pick your model: 1-RNN, 2-LSTM, 3-FFNN, 4-Transformer"))

if choice == 1:

  # fixing every word's embedding size to be 32
  embd_len = 32
  vocab_size =500

  # Creating a RNN model
  model = Sequential(name="Simple_RNN")
  model.add(Embedding(vocab_size,
                          embd_len,
                          input_length=df_train.shape[1]))

  # In case of a stacked(more than one layer of RNN)
  # use return_sequences=True
  model.add(SimpleRNN(128,
                          activation='tanh',
                          return_sequences=True))
  model.add(SimpleRNN(128,
                          activation='tanh',
                          return_sequences=True))

  model.add(SimpleRNN(128,
                          activation='tanh',
                          return_sequences=True))

  model.add(SimpleRNN(128,
                          activation='tanh',
                          return_sequences=False))

  model.add(Dense(3, activation='sigmoid'))

  # printing model summary
  print(model.summary())

  # Compiling model
  model.compile(
      loss="binary_crossentropy",
      optimizer='adam',
      metrics=['accuracy']
  )
  # Printing model score on test data
  # print()
  # print("Simple_RNN Score---> ", model.evaluate(x_test, y_test, verbose=0))

elif choice == 2:


    

  # fixing every word's embedding size to be 32
  embd_len = 32
  vocab_size =500

  # Creating a RNN model
  model = Sequential(name="LSTM")
  model.add(Embedding(vocab_size,
                          embd_len,
                          input_length=df_train.shape[1]))

  # In case of a stacked(more than one layer of RNN)
  # use return_sequences=True
  model.add(LSTM(128, activation='tanh', return_sequences=True))
  model.add(LSTM(128, activation='tanh', return_sequences=True))

  model.add(LSTM(128, activation='tanh', return_sequences=True))

  model.add(LSTM(128, activation='tanh', return_sequences=False))

  model.add(Dense(3, activation='sigmoid'))

  # printing model summary
  print(model.summary())

  # Compiling model
  model.compile(
      loss="binary_crossentropy",
      optimizer='adam',
      metrics=['accuracy']
  )
  # Printing model score on test data
  # print()
  # print("Simple_RNN Score---> ", model.evaluate(x_test, y_test, verbose=0))


elif choice == 3:

  # fixing every word's embedding size to be 32
  embd_len = 32
  vocab_size =500

  # Creating a RNN model
  model = Sequential(name="FFNN")
  model.add(Embedding(vocab_size,
                          embd_len,
                          input_length=df_train.shape[1]))

  # In case of a stacked(more than one layer of RNN)
  # use return_sequences=True
  model.add(Dense(128,
                          activation='tanh'))
  model.add(Dense(128,
                          activation='tanh'))

  model.add(Dense(128,
                          activation='tanh'))

  model.add(Dense(128,
                          activation='tanh'))
  model.add(Flatten())

  model.add(Dense(3, activation='sigmoid'))

  # printing model summary
  print(model.summary())

  # Compiling model
  model.compile(
      loss="binary_crossentropy",
      optimizer='adam',
      metrics=['accuracy']
  )
  # Printing model score on test data
  # print()
  # print("Simple_RNN Score---> ", model.evaluate(x_test, y_test, verbose=0))


elif choice == 4:


  embed_dim = 32  # Embedding size for each token
  num_heads = 2  # Number of attention heads
  ff_dim = 32  # Hidden layer size in feed forward network inside transformer

  inputs = Input(shape=(max_len_vec,))
  embedding_layer = TokenAndPositionEmbedding(max_len_vec, vocab_size, embed_dim)
  x = embedding_layer(inputs)
  transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
  x = transformer_block(x)
  x = transformer_block(x)
  x = transformer_block(x)
  x = transformer_block(x)
  x = transformer_block(x)
  x = GlobalAveragePooling1D()(x)
  x = Dropout(0.1)(x)
  x = Dense(20, activation="relu")(x)
  x = Dropout(0.1)(x)
  outputs = Dense(3, activation="softmax")(x)

  model = Model(inputs=inputs, outputs=outputs)

else:
  print('wrong input')



Pick your model: 1-RNN, 2-LSTM, 3-FFNN, 4-Transformer4


In [134]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])


In [135]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 81)]         0           []                               
                                                                                                  
 token_and_position_embedding_1  (None, 81, 32)      63392       ['input_2[0][0]']                
  (TokenAndPositionEmbedding)                                                                     
                                                                                                  
 transformer_block_1 (Transform  (None, 81, 32)      10656       ['token_and_position_embedding_1[
 erBlock)                                                        0][0]',                          
                                                                  'transformer_block_1[0][0]

In [136]:

history = model.fit(df_train, y_train, epochs=10, 
                    validation_data=(df_test, y_test)
                   )

Epoch 1/2
40/40 [==============================] - 5s 36ms/step - loss: 0.9556 - accuracy: 0.5627 - val_loss: 0.8946 - val_accuracy: 0.6140
Epoch 2/2
40/40 [==============================] - 1s 21ms/step - loss: 0.7843 - accuracy: 0.6748 - val_loss: 0.7165 - val_accuracy: 0.7316


# Prediction

In [55]:
dataset_test = pd.read_csv('project2_test_data.txt', delimiter= '\r\n' , header = None, names=['text'])

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [139]:
dataset_test.head()

,text
0,The company 's operating income ( EBIT ) total...
1,"Glaston 's own glass processing unit , Tamglas..."
2,"In September alone , the market declined by 10..."
3,Goodwill and other intangible assets account f...
4,Major Order in India Comptel Corporation has r...


In [140]:
df2 = dataset_test.copy()

In [141]:
# removing special characters
df2 = df2.replace('\*', ' ')
df2 = df2.replace('\xc2', ' ', regex=True)
df2 = df2.replace('\xf1', ' ', regex=True)
df2 = df2.replace('\xc1', ' ', regex=True)
df2 = df2.replace('[^a-zA-Z]', ' ')
# df = df.replace('"+"', ' ', regex=True)
# df = df.replace('"+"', ' ', regex=True)


In [142]:
df2

,text
0,The company 's operating income ( EBIT ) total...
1,"Glaston 's own glass processing unit , Tamglas..."
2,"In September alone , the market declined by 10..."
3,Goodwill and other intangible assets account f...
4,Major Order in India Comptel Corporation has r...
...,...
448,"Cargo traffic fell 1 % year-on-year to 8,561 t..."
449,The value of the orders is over EUR 10mn .
450,"The sellers include 40 shareholders , includin..."
451,"In Sweden , operating profit for the period un..."


In [143]:
df2['text'] = var3.texts_to_sequences(df2['text'])

In [144]:
df2

,text
0,"[1, 13, 11, 33, 322, 811, 172, 6, 39, 39, 10, ..."
1,"[790, 11, 347, 1267, 1037, 142, 1267, 1037, 12..."
2,"[3, 112, 3222, 1, 69, 759, 19, 94, 30, 71, 23,..."
3,"[1591, 4, 110, 947, 3773, 8, 160, 30, 39, 28, ..."
4,"[302, 107, 3, 1778, 558, 88, 41, 543, 7, 423, ..."
...,...
448,"[461, 401, 193, 17, 23, 16, 23, 5, 49, 412, 3,..."
449,"[1, 83, 2, 1, 288, 12, 168, 6, 1569]"
450,"[1, 1485, 185, 213, 335, 220, 2859, 169, 214, ..."
451,"[3, 208, 33, 14, 8, 1, 35, 192, 1236, 136, 6, ..."


In [146]:
df2['text'] = tf.keras.preprocessing.sequence.pad_sequences(df2['text'], maxlen=max_len_vec)

In [147]:
df2

,text
0,0
1,0
2,0
3,0
4,0
...,...
448,0
449,0
450,0
451,0


In [153]:
predicts = model.predict(df2['text'])

15/15 [==============================] - 0s 13ms/step


In [154]:
print(predicts)

[[0.00629144 0.9334995  0.06020897]
 [0.00629144 0.9334995  0.06020897]
 [0.00629144 0.9334995  0.06020897]
 ...
 [0.00629144 0.9334995  0.06020899]
 [0.00629144 0.9334995  0.06020899]
 [0.00629144 0.9334995  0.06020899]]


In [155]:
lst_pred = []

for element in predicts:
  element = np.argmax(element)
  lst_pred.append(element)

In [174]:
final_pred = le.inverse_transform(lst_pred)


# Predicted class labels


In [175]:
final_pred

array(['neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
       'neutral', 'neutral', 'neutral', 'neutral', 

In [176]:
with open('class_labels.txt', 'w+') as f:
  #final_pred = str(final_pred)
  for pred in final_pred:
    f.write(pred + '\n')